In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [95]:
data = pd.read_csv("data_60_points.csv")
final_df = pd.read_csv("real_trainable_data.csv")

In [97]:
final_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,Target
0,24.292663,19.482136,8.624346,-0.932498,-0.395269,-0.455120,0.855522,0.265530,0.903650,0.375534,6.494048,10.806570,16.298106,22.642826,30.166633,32.797640,35.965448,36.856227,38.217799,40.515852,44.777485,47.852875,47.984674,44.949001,39.392795,41.983571,41.751005,39.821448,39.306901,35.898327,31.267594,28.991488,29.493162,27.268908,24.621208,19.164740,9.364607,-2.791931,-2.903897,-6.950359,-5.469834,-5.304405,-6.784444,-11.026843,-14.940937,-18.745704,-16.041380,-13.575005,-11.913996,-11.406204,-13.584742,-17.775369,-23.828478,-29.688440,-27.825962,-28.735885,-25.563278,-23.956415,-24.071713,-25.723195,23.887877,19.181469,7.703152,-0.299300,-0.602985,-1.538617,1.394858,1.038005,0.064751,-0.604627,4.885005,13.327236,18.638853,20.475003,24.872449,27.910945,30.766082,33.627088,33.756718,41.061445,46.707569,48.808433,48.281197,45.585899,40.289640,38.265429,40.123781,40.566784,38.143946,32.827875,29.347441,28.867881,28.963388,27.365379,22.617756,16.638018,9.851901,-3.661786,-4.820651,-9.384957,-10.326928,-9.248978,-10.756186,-13.218735,-16.225137,-19.230096,-17.075962,-13.276833,-11.900167,-12.169421,-13.894676,-17.769490,-24.388912,-27.433080,-27.771646,-26.974928,-24.615398,-23.666156,-24.486484,-26.730907,24.566455,19.962520,8.848087,-0.763101,-0.582455,-3.345621,-4.188401,-4.568384,-3.120046,-0.108741,5.084797,11.905776,16.501846,19.086502,25.892442,32.645678,37.600500,39.718644,39.254302,40.421854,45.386670,47.860203,47.758835,44.971354,39.621061,40.075795,39.775275,41.031486,39.255390,35.103430,33.212198,30.152583,31.204244,29.944917,25.263661,17.940899,10.494879,-1.343950,-4.056530,-6.456099,-5.280097,-5.482480,-8.627317,-13.419672,-15.561671,-17.848400,-16.996055,-13.943277,-12.615044,-12.641014,-13.874491,-16.950376,-22.810055,-29.546383,-28.165966,-29.040384,-26.217174,-24.865166,-25.613848,-25.122706,22.911102,18.271054,6.775453,-1.641572,-2.169056,-4.451217,-0.232341,0.459793,-1.303518,-3.035776,2.724729,10.830275,14.428922,20.836431,26.936384,29.802237,30.920108,29.222181,29.825365,39.045233,44.991545,47.294873,46.790536,43.632752,39.732283,42.429897,42.140096,38.443610,34.656696,33.470449,30.128382,29.033831,29.208666,27.375852,22.74

## ----- testing ----

In [5]:
#This onwards is testing since CNN (convolutional neural networks) require certain dimensions and the data above is not
#how it is required. I have not yet got that part correctly. Maybe another try can be with some other technique, like SVM
#other any other with the data above, or the required trasformations.

In [55]:
testing = np.array(data)
channels = 8
data_points = 60
rows = int(testing.shape[0]/channels)

testing = testing.reshape(rows,channels,data_points)
testing.shape

(24480, 8, 60)

In [56]:
#testing = np.transpose(testing,(0,2,1)) #swapping the dimensions of the first and second dimension
#testing = np.moveaxis(testing,1,2)
testing[0,0,:]

array([ 24.29266315,  19.48213606,   8.62434618,  -0.93249795,
        -0.395269  ,  -0.45511959,   0.85552233,   0.26553007,
         0.90364979,   0.37553439,   6.49404827,  10.80657046,
        16.29810641,  22.64282591,  30.16663306,  32.79763987,
        35.96544763,  36.85622728,  38.21779915,  40.51585161,
        44.77748515,  47.85287506,  47.98467419,  44.94900146,
        39.39279476,  41.98357109,  41.75100534,  39.82144845,
        39.30690095,  35.89832734,  31.26759443,  28.99148835,
        29.49316214,  27.26890834,  24.621208  ,  19.16474037,
         9.36460695,  -2.79193057,  -2.9038967 ,  -6.95035885,
        -5.4698337 ,  -5.30440495,  -6.7844437 , -11.02684303,
       -14.94093665, -18.74570367, -16.04137975, -13.575005  ,
       -11.91399572, -11.40620428, -13.58474193, -17.7753692 ,
       -23.82847803, -29.68844043, -27.82596152, -28.73588454,
       -25.56327793, -23.95641506, -24.07171348, -25.72319523])

In [11]:
testing.shape

(24480, 60, 8)

In [12]:
testing[0,0,:]

array([24.29266315, 13.98955519, 28.22633846, 10.27599949, 25.7011592 ,
        8.78540736, 25.63732919,  9.94014041])

# Train Test Split

In [13]:
inputs = testing

In [14]:
inputs

array([[[ 24.29266315,  13.98955519,  28.22633846, ...,   8.78540736,
          25.63732919,   9.94014041],
        [ 19.48213606,  10.53884378,  22.92692172, ...,   3.43229632,
          20.01987867,   3.50816165],
        [  8.62434618,   4.35775745,  12.56838637, ...,  -3.57635655,
           9.04231123,  -9.03445603],
        ...,
        [-23.95641506, -32.90878499, -18.74202104, ..., -37.07345309,
         -19.19802768, -29.79290376],
        [-24.07171348, -29.10308578, -18.91035331, ..., -31.40481203,
         -17.44934579, -28.08923238],
        [-25.72319523, -26.50364474, -16.05159439, ..., -28.95965915,
         -16.75149523, -28.63564027]],

       [[ 19.89690772,  13.59079517,  21.76846978, ...,  15.62146155,
          25.35015354,  19.03803218],
        [ 13.44733366,   5.29455776,  16.08370541, ...,  11.26126993,
          19.06618013,  14.74662409],
        [  1.75833183,  -8.4935791 ,   4.77559825, ...,   5.48320985,
           7.83089717,   3.67842121],
        ...,


In [15]:
targets = final_df.iloc[:,final_df.columns.size-1]
targets = np.array(targets.astype(int))
targets

array([ 1,  2,  3, ..., 10, 11, 12])

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size = 0.1, random_state = 42)

In [17]:
#from sklearn.preprocessing import StandardScaler

#maybe later

#scaler = StandardScaler()
#x_train = scaler.fit_transform(x_train)
#x_test = scaler.transform(x_test)

In [ ]:
#Try: LSTM in here or from the following link in matlab: 

# https://www.mathworks.com/help/deeplearning/ug/classify-sequence-data-using-lstm-networks.html